In [2]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
GPU (s):
0.0404721899976721


2023-05-03 01:16:33.524132: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-03 01:16:33.524327: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-03 01:16:33.524440: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-03 01:16:33.524600: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-03 01:16:33.524718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']

In [5]:
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Load the CIFAR-100 dataset
data_path = 'cifar-100-python/'
train_data_100 = unpickle(data_path + 'train')[b'data']
train_data_100 = train_data_100.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data_100 = unpickle(data_path + 'test')[b'data']
test_data_100 = test_data_100.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels_100 = np.array(unpickle(data_path + 'train')[b'fine_labels'])
test_labels_100 = np.array(unpickle(data_path + 'test')[b'fine_labels'])

In [6]:
# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Print the number of features, classes, and training samples for CIFAR-100
print("\nCIFAR-100")
print("Number of features:", train_data_100.shape[1:])
print("Number of classes:", len(np.unique(train_labels_100)))
print("Number of training samples:", train_data_100.shape[0])

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000

CIFAR-100
Number of features: (32, 32, 3)
Number of classes: 100
Number of training samples: 50000


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
train_data_100, train_labels_100 = shuffle(train_data_100, train_labels_100, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)
test_data_100, test_labels_100 = shuffle(test_data_100, test_labels_100, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0
x_train_100 = train_data_100.astype('float32') / 255.0
x_test_100 = test_data_100.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)
y_train_100 = to_categorical(train_labels_100, num_classes=100)
y_test_100 = to_categorical(test_labels_100, num_classes=100)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

In [8]:
#Build ResNet-50 model
base_model = ResNet50(include_top=False, weights=None, input_shape=(32, 32, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model_10 = Model(inputs=base_model.input, outputs=predictions)

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu.{epoch:02d}.h5', save_freq='epoch')

learning_rate = 0.001

# Define a learning rate schedule function
def lr_schedule(epoch):
    """
    Returns a learning rate based on the epoch number.
    """
    learning_rate = 0.001
    if (epoch>1 and epoch<=3):
        learning_rate = 0.00075
    if (epoch>3 and epoch<=5):
        learning_rate = 0.0005
    if (epoch>5 and epoch<=7):
        learning_rate = 0.00025
    if (epoch>7 and epoch<=9):
        learning_rate = 0.000125
    if (epoch>9 and epoch<=11):
        learning_rate = 0.0000625
    if (epoch>11 and epoch<=13):
        learning_rate = 0.00003125
    if (epoch>13 and epoch<= 15):
        learning_rate = 0.00001575
    if (epoch>15):
        learning_rate = 0.000007375
    return learning_rate

# Create a LearningRateScheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

model_10.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
# Model Fit
model_10.fit(it_train, steps_per_epoch=steps, epochs=25, validation_data=(x_test, y_test), callbacks=[lr_scheduler,checkpoint_callback])

Epoch 1/25
781/781 [==============================] - 25s 32ms/step - loss: 1.4341 - accuracy: 0.4784 - val_loss: 1.6464 - val_accuracy: 0.3920 - lr: 0.0010
Epoch 2/25
781/781 [==============================] - 24s 31ms/step - loss: 1.2852 - accuracy: 0.5405 - val_loss: 1.6181 - val_accuracy: 0.4538 - lr: 0.0010
Epoch 3/25
781/781 [==============================] - 25s 32ms/step - loss: 1.2506 - accuracy: 0.5536 - val_loss: 3.0872 - val_accuracy: 0.3677 - lr: 7.5000e-04
Epoch 4/25
781/781 [==============================] - 25s 31ms/step - loss: 1.1465 - accuracy: 0.5916 - val_loss: 1.9163 - val_accuracy: 0.4807 - lr: 7.5000e-04
Epoch 5/25
781/781 [==============================] - 25s 32ms/step - loss: 0.9990 - accuracy: 0.6515 - val_loss: 1.2009 - val_accuracy: 0.5980 - lr: 5.0000e-04
Epoch 6/25
781/781 [==============================] - 25s 32ms/step - loss: 1.0823 - accuracy: 0.6154 - val_loss: 1.0325 - val_accuracy: 0.6445 - lr: 5.0000e-04
Epoch 7/25
781/781 [======================

In [11]:
# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 3s 9ms/step - loss: 0.6506 - accuracy: 0.7767
Test accuracy: 0.7767000198364258


In [ ]:
# create data generator
datagen_100 = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
it_train_100 = datagen.flow(x_train_100, y_train_100, batch_size=64)
# fit model
steps = int(x_train_100.shape[0] / 64)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten

model_100 = Sequential()
model_100.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model_100.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_100.add(MaxPooling2D((2, 2)))
model_100.add(Dropout(0.2))
model_100.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_100.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_100.add(MaxPooling2D((2, 2)))
model_100.add(Dropout(0.3))
model_100.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_100.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_100.add(MaxPooling2D((2, 2)))
model_100.add(Dropout(0.4))
model_100.add(Flatten())
model_100.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_100.add(Dropout(0.5))
model_100.add(Dense(100, activation='softmax'))

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_100_gpu.{epoch:02d}.h5', save_freq='epoch')

learning_rate = 0.0001

# Define a learning rate schedule function
def lr_schedule(epoch):
    """
    Returns a learning rate based on the epoch number.
    """
    learning_rate = 0.001
    if (epoch>1 and epoch<=3):
        learning_rate = 0.00075
    if (epoch>3 and epoch<=5):
        learning_rate = 0.0005
    if (epoch>5 and epoch<=7):
        learning_rate = 0.00025
    if (epoch>7 and epoch<=9):
        learning_rate = 0.000125
    if (epoch>9 and epoch<=11):
        learning_rate = 0.0000625
    if (epoch>11 and epoch<=13):
        learning_rate = 0.00003125
    if (epoch>13 and epoch<= 15):
        learning_rate = 0.00001575
    if (epoch>15):
        learning_rate = 0.000007375
    return learning_rate

# Create a LearningRateScheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

model_100.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])

# Model Fit
model_100.fit(it_train_100, steps_per_epoch=steps, epochs=200, validation_data=(x_test_100, y_test_100), callbacks=[checkpoint_callback])

Epoch 1/200
781/781 [==============================] - 15s 18ms/step - loss: 4.6402 - accuracy: 0.0100 - val_loss: 4.6052 - val_accuracy: 0.0110
Epoch 2/200
781/781 [==============================] - 14s 18ms/step - loss: 4.6042 - accuracy: 0.0100 - val_loss: 4.5970 - val_accuracy: 0.0123
Epoch 3/200
781/781 [==============================] - 16s 20ms/step - loss: 4.5304 - accuracy: 0.0220 - val_loss: 4.4067 - val_accuracy: 0.0414
Epoch 4/200
781/781 [==============================] - 14s 18ms/step - loss: 4.3638 - accuracy: 0.0373 - val_loss: 4.2061 - val_accuracy: 0.0746
Epoch 5/200
781/781 [==============================] - 14s 17ms/step - loss: 4.2109 - accuracy: 0.0570 - val_loss: 4.0179 - val_accuracy: 0.0974
Epoch 6/200
781/781 [==============================] - 13s 17ms/step - loss: 4.0810 - accuracy: 0.0723 - val_loss: 3.8621 - val_accuracy: 0.1278
Epoch 7/200
781/781 [==============================] - 15s 19ms/step - loss: 3.9738 - accuracy: 0.0887 - val_loss: 3.7737 - val_ac